In [1]:
import pandas as pd
import numpy as np
import pymssql
from fuzzywuzzy import fuzz

### Import Data

In [2]:
df = pd.read_csv('./meme_cleaning.csv')

In [29]:
df

,Tweet_ID,Posted_By,Twitter_Handle,Tweet,Created_At,No_of_Likes,No_of_Retweets,No_of_Followers,User_Location,Image_URL,hashtags
0,1,furry_irl,rFurry_Irl,Furry_irl | https://t.co/vPgHv1R6W6 #memes #fu...,2021-03-06 07:13:12.000,0,0,501,NaN,http://pbs.twimg.com/media/Evx1QhLWYAArjHu.jpg,"[#memes, #furry]"
1,2,SidenoteSuplex,SidenoteSuplex,The Speculation continues...#AEW #AEWRevolutio...,2021-03-06 07:06:44.000,0,0,233,NaN,http://pbs.twimg.com/media/EvxzdHqXcAchlkN.jpg,"[#aew, #aewrevolution, #memes, #wrestlingtwitt..."
2,3,FunnyThings,funny_things127,When you’re dead inside but your best friend n...,2021-03-06 07:06:31.000,0,0,1508,NaN,http://pbs.twimg.com/media/EvxzubJXEAAd4_-.jpg,[#memes]
3,4,????????????,chic_aya,POV: Your black cat has a mid '9 lives' crisis...,2021-03-06 07:03:05.000,1,0,138,NaN,http://pbs.twimg.com/tweet_video_thumb/Evxygjw...,"[#memes, #caturday, #catsoftwitter]"
4,5,Tell Tales Entertainment,TellTalesOnline,We hate being so savage?? #meme #memes #kimkar...,2021-03-06 07:00:19.000,2,0,410,NaN,http://pbs.twimg.com/media/EvxyTy6WgAALDAb.png,"[#meme, #memes, #kimkardashian]"
...,...,...,...,...,...,...,...,...,...,...,...
1324,1325,Dank Meme Dealer ????,DankRedditBot,No way it's nearly been 5 years #dankmemes #me...,2021-03-04 08:50:28.000,7,1,9420,YourMomsHouse,http://pbs.twimg.com/media/Evn4V3OXEAg4YZE.jpg,"[#dankmemes, #memes, #memesdaily]"
1325,1326,Daily dose of memes to make you laugh,MemesUniteUs,ik im late but no horny\n#memes #memesdaily ht...,2021-03-04 08:50:27.000,0,0,689,USM - United States of Memes,http://pbs.twimg.com/media/Evn4VnMXEBoJHKh.png,"[#memes, #memesdaily]"
1326,1327,Daily dose of memes to make you laugh,MemesUniteUs,belt go snap\n#memes #memesdaily https://t.co/...,2021-03-04 08:50:25.000,0,0,689,USM - United States of Memes,http://pbs.twimg.com/media/Evn4VHuWgAAHpxW.jpg,"[#memes, #memesdaily]"
1327,1328,Daily dose of memes to make you laugh,MemesUniteUs,health: 8/100\n#memes #memesdaily https://t.co...,2021-03-04 08:50:23.000,1,0,689,USM - United States of Memes,http://pbs.twimg.com/media/Evn4Ul3WgAANwbE.jpg,"[#memes, #memesdaily]"


In [25]:
df.Tweet[df.Tweet.str.contains('#Wanda')]

131     #WandaVision spoilers!!!\n-\n-\n-\n-\n-\n-\n-\...
177     Wanda when the Westview residents start asking...
414     Is this how memes work? #WandaVision #Spoilers...
729     I'm not ready for tomorrow! Make sure to #foll...
842     Has this ever happened to you? It has to me. #...
1033    Is it \n\n#MEMES #exammemes #WandaVisionFinale...
Name: Tweet, dtype: object

### Getting Hashtags to Play Nice

In [3]:
#Extract all words that begin with # and turn the results into a dataframe
temp = df['Tweet'].str.lower().str.extractall(r"(#\w+)")
temp.columns = ['unnamed']
# Convert the multiple hashtag values into a list
temp = temp.groupby(level = 0)['unnamed'].apply(list)
# Save the result as a feature in the original dataset
df['hashtags'] = temp

In [4]:
# search through our new hashtag feature and find unique hashtags; place them into the dictionary in the form {hashtag: number of instances}
uniques_count = {}
for value in df['hashtags']:
    for word in value:
        # If the hashtag is already in the dictionary, increment the value by 1
        if word in uniques_count:
            uniques_count[word] += 1
        else:
            uniques_count[word] = 1

In [5]:
pd.Series(uniques_count).sort_values(ascending=False).head()

#memes         1333
#memesdaily     186
#dankmemes      134
#furry          120
#funny          103
dtype: int64

In [6]:
hashtags = pd.DataFrame(uniques_count.keys(), columns=['hashs'])
hashtags

,hashs
0,#memes
1,#furry
2,#aew
3,#aewrevolution
4,#wrestlingtwitter
...,...
833,#macncheese
834,#balls
835,#rickrollmemes
836,#pursuitofhappiness


In [7]:
len(uniques_count.keys())

838

### Topics

In [8]:
trends = pd.read_csv('./trends.csv')
trends_ = list(set(pd.concat((trends.iloc[:,1], trends.iloc[:,2]), axis = 0)))

In [9]:
# def apply_list(rowval):
#     trendlist = []
#     for word in rowval:
#         for trend in trends_:
#             if fuzz.ratio(word, trend) > 60:
#                 trendlist.append(trend)
#     return trendlist

# df['Tweet'].apply(apply_list)

### Sentiment

In [10]:
sent = pd.read_csv('./sentiments.csv')

In [11]:
sent['Labels'].fillna('None', inplace = True)
collect = sent['Labels'].str.lower().str.split()

In [12]:
uniques_count = {}
for value in collect:
    for word in value:
        # If the hashtag is already in the dictionary, increment the value by 1
        if word in uniques_count:
            uniques_count[word] += 1
        else:
            uniques_count[word] = 1

In [13]:
count = pd.DataFrame([uniques_count.keys(), uniques_count.values()]).T
count.columns = ['labels','count']
count.sort_values(by = 'count', ascending = False).head(10)

,labels,count
7,font,855
1,gesture,328
3,happy,305
5,art,283
84,cartoon,272
77,organism,261
82,human,210
15,product,210
155,facial,202
79,event,183


## Insert into database

In [14]:
# server = 'imt563-winter21.database.windows.net'
# database = 'Team1'
# username = 'imt563'
# password = 'SuperSafePassword1234'

# conn = pymssql.connect(server, username, password, database)
# cursor = conn.cursor()

### hashtag

In [15]:
# for index, row in hashtags.iterrows():
#     cursor.execute("INSERT INTO dbo.hashtag(hashtag_text) values(%s)" % ("\'" + row[0] + "\'"))
#     print(index, row[0])
# conn.commit()
# conn.close()

### user_profile

In [16]:
user_profile = df[['Twitter_Handle','No_of_Followers']].drop_duplicates(subset = ['Twitter_Handle'])

In [17]:
for index, row in user_profile.iterrows():
    cursor.execute("INSERT INTO dbo.user_profile(profile_name, follower_count) values(%s, %d)" % ("\'" + row.Twitter_Handle + "\'",row.No_of_Followers))
    print(index, row.Twitter_Handle, row.No_of_Followers)
conn.commit()
conn.close()

NameError: name 'cursor' is not defined

### photo

In [ ]:
photo = sent[['Image_URL','vaderSentimentScores','vaderSentiment']]

In [ ]:
# for index, row in photo.iterrows():
#     cursor.execute("INSERT INTO dbo.photo(photo_url, vader_score, vader_sentiment) values(%s, %f, %s)" % ("\'" + row[0] + "\'", row[1],"\'" + row[2] + "\'"))
#     print(index, row[0])
# conn.commit()
# conn.close()

### sentiment

In [ ]:
sentiment = pd.DataFrame(uniques_count.keys())

In [ ]:
for index, row in sentiment.iterrows():
    cursor.execute("INSERT INTO dbo.sentiment(sentiment_name) values(%s)" % ("\'" + row[0] + "\'"))
    print(index, row[0])
conn.commit()
conn.close()

### temp - Focus on interaction between profiles, photos, and post

In [ ]:
temp = df.iloc[:,2:10].drop(['User_Location','No_of_Followers'], axis = 1)
temp['Created_At'] = pd.to_datetime(temp['Created_At']).dt.date.astype(str)
temp['Tweet'] = temp['Tweet'].str.replace("'",'').str.lower()

In [ ]:
for index, row in temp.iterrows():
    cursor.execute("INSERT INTO dbo.temp(profile_name, post_date, photo_url, tweet_text, like_count, retweet_count) values(%s, %s, %s, %s, %d, %d)" % ("\'" + row[0] + "\'", row[2], "\'" + row[5] + "\'", "\'" + row[1] + "\'", row[3], row[4]))
conn.commit()
conn.close()

### temp - Focus on interaction between post and hashtag

In [ ]:
temp = df[['Tweet','hashtags']]
temp['Tweet'] = temp['Tweet'].str.replace("'",'').str.lower()
temp = temp.explode('hashtags')

In [ ]:
for index, row in temp.iterrows():
    cursor.execute("INSERT INTO dbo.temp(tweet, hashtag) values(%s, %s)" % ("\'" + row[0] + "\'","\'" + row[1] + "\'"))
conn.commit()
conn.close()

### temp - Focus on interaction between sentiments and photos

In [ ]:
temp = sent[['Image_URL','Labels']]
temp['Labels'] = temp['Labels'].str.lower().str.split()
temp = temp.explode('Labels')

In [ ]:
for index, row in temp.iterrows():
    cursor.execute("INSERT INTO dbo.temp(photo_url, label) values(%s, %s)" % ("\'" + row[0] + "\'","\'" + row[1] + "\'"))
conn.commit()
conn.close()

# EDA

Let's take a quick look at our data:

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().sum().sum()

There are a little over 1300 observations and 11 features. I found 575 missing values, but they are all concentrated in the 'User_Location' feature, which will not be overly helpful in predicting engagement.

In [ ]:
X = df[['hashtags','Image_URL']]

In [ ]:
y = (df['No_of_Retweets']+1)/(df['No_of_Followers']+1)